# PROGRAMOWANIE NISKOPOZIOMOWE
## laboratorium 5 zadanie 7

Porównaj eksperymentalnie złożoność trzech algorytmów sortowania: bąbelkowego, stogowego i szybkiego.
Użyj/zdefiniuj w tym celu trzy różne funkcje sortowania ciągu liczb całkowitych (bąbelkowe, stogowe
i szybkie), a także uniwersalną funkcję pomiaru (pomierz) do realizacji pomiarów dla wskazanego ciągu liczb
i wskazanego algorytmu sortowania (parametr przekazany przez wskaźnik funkcji!). Przeprowadź 1000
losowań ciągów 300-elementowych, o wartościach z przedziału <0;1000>. Każdy wylosowany ciąg posortuj
trzema algorytmami. Porównaj uśrednione czasy sortowania. Wyprowadź te czasy na standardowe wyjście,
opatrując odpowiednim komentarzem. Wyciągnij wnioski i umieść je na końcu programu.
Oto format danych oczekiwanych na standardowym wyjściu:
Dane:
BA t1
ST t2
SZ t3
gdzie t1 oznacza średni czas sortowania ciągu 300-elementowego metodą bąbelkową, t2 – średni czas
sortowania stogowego, zaś t3 – średni czas sortowania szybkiego. Oto sugerowana postać nagłówka funkcji
przeznaczonej do pomiarów:

In [ ]:
int pomierz(int *Dane, int Rozmiar, void(*sort)(int[], int))

Rozwiąż zadanie przy użyciu stosownych, odpowiednio sparametryzowanych funkcji, w tym trzech funkcji
sortowania ciągu. Pamiętaj, by za każdym razem porównywać czasy sortowania tego samego ciągu.

**Wykorzystywane biblioteki**

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <chrono>
using namespace std::chrono;

**Deklaracje funkcji**

W tym zadaniu wykorzystane zostaly funkcje z zadania 6: losowaLiczba, wypelnijLosowymi, wygenerujTablice oraz sortowanieBabelkoweTablica.

In [ ]:
int losowaLiczba(int min, int max);
void wypelnijLosowymi(int* tablica, int rozmiar = 300, int min = 0, int max = 1000);
int* wygenerujTablice(int* losowe, int rozmiar = 300);
void sortowanieBabelkoweTablica(int* tablica, int rozmiar = 300);
double pomiar(int* tablica, int rozmiar, void(*sort)(int*, int));
void stoguj(int* tablica, int rozmiar, int i);
void sortowanieStogoweTablica(int* tablica, int rozmiar = 300);
int sortowanieSzybkieTablica(int* tablica, int lewa, int prawa, int rozmiar = 300);
void sortowanieSzybkieTablicaInicjacja(int* tablica, int rozmiar);

**Definicje funkcji**

Funkcja losujaca liczbe z zakresu <min;max>

In [ ]:
int losowaLiczba(int min, int max) {
	return (rand() % (max - min + 1)) + min;
}

Funkcja wypelniajaca tablice o podanym rozmiarze (domyslnie 300) liczbami z zakresu od min do max (domyslnie od 0 do 1000)

In [ ]:
void wypelnijLosowymi(int* tablica, int rozmiar = 300, int min = 0, int max = 1000) {
	for (int i = 0; i < rozmiar; i++) {
		tablica[i] = losowaLiczba(min, max);
	}
}

Funkcja generujaca tablice dynamiczna o podanym rozmiarze (domyslnie 300) wypelniona liczbami z tablicy liczb losowych z analogicznych pozycji.

In [ ]:
int* wygenerujTablice(int* losowe, int rozmiar = 300) {
	int* tablica = (int*)malloc(rozmiar * sizeof(int));
	for (int i = 0; i < rozmiar; i++) {
		tablica[i] = losowe[i];
	}
	return tablica;
}

Funkcja sotujaca tablice przy uzyciu algorytmu sortowania babelkowego. Jest to najwolniejszy z testowanych w tym zadaniu algorytmow sortowania, gdyz jego zlozonosc wynosi O(n^2) co wynika z faktu, iz n razy musi przejsc przez (n minus iteracja)
elementow

In [ ]:
void sortowanieBabelkoweTablica(int* tablica, int rozmiar = 300) {
	for (int i = 0; i < rozmiar - 1; i++) {
		for (int j = 0; j < rozmiar - i - 1; j++) {
			if (tablica[j] > tablica[j + 1]) {
				int lewa = tablica[j];
				int prawa = tablica[j + 1];
				tablica[j] = prawa;
				tablica[j + 1] = lewa;
			}
		}
	}
}

Funkcje niewystepujace w zadaniach poprzednich

Funkcja realizujaca pomiary dla wskazanego ciągu liczb i wskazanego algorytmu sortowania.

In [ ]:
double pomiar(int* tablica, int rozmiar, void(*sort)(int*, int)) {
	auto start1 = steady_clock::now();
	sort(tablica, rozmiar);
	auto stop1 = steady_clock::now();
	return (double)duration_cast<microseconds>(stop1 - start1).count();
}


Zaraz po algorytmie babelkowym znajduje sie sortowanie przez kopcowanie, ktore majac ta sama zlozonosc co sortowanie szybkie,
czyli O(n log(n)), wymaga w każdej iteracji wykonania wiekszej ilosci operacji co wplywa na to, ze jest on wolniejszy.

In [ ]:
void stoguj(int* tablica, int rozmiar, int i) {
	int lewa = i * 2 + 1;
	int prawa = i * 2 + 2;
	int doZmiany = -1;
	if (prawa < rozmiar && lewa < rozmiar) {
		if (tablica[prawa] > tablica[i] || tablica[lewa] > tablica[i]) {
			doZmiany = (tablica[lewa] > tablica[prawa]) ? lewa : prawa;
		}
	}
	else if (lewa < rozmiar && tablica[lewa] > tablica[i]) doZmiany = lewa;
	if (doZmiany != -1) {
		int tmp = tablica[i];
		tablica[i] = tablica[doZmiany];
		tablica[doZmiany] = tmp;
		stoguj(tablica, rozmiar, doZmiany);
	}
}

In [ ]:
void sortowanieStogoweTablica(int* tablica, int rozmiar = 300) {
	int* nowa = (int*)malloc(rozmiar * sizeof(int));
	int nowaRozmiar = 0;
	int n = rozmiar;
	for (int j = n / 2; j >= 0; j--) {
		stoguj(tablica, n, j);
	}
	for (int i = n; i > 0; i--) {
		int tmp = tablica[n - 1];
		tablica[n - 1] = tablica[0];
		tablica[0] = tmp;
		nowa[rozmiar - 1 - nowaRozmiar++] = tablica[--n];
		stoguj(tablica, n, 0);
	}
	for (int i = 0; i < rozmiar; i++) {
		tablica[i] = nowa[i];
	}
}

Sortowanie szybkie jest najszybszym algorytmem dla tego zestawu danych, choc w tej implementacji radzi on sobie najlepiej dla zestawu danych losowych nie monotonicznych co sprawia, ze jest on jedynym z tych 3 algorytmow, ktory wykazuje duza
roznice czasowa w zaleznosci od danych wejsciowych, co jest nieodczuwalne dla wykonanych testow.

In [ ]:
int sortowanieSzybkieTablica(int* tablica, int lewa, int prawa, int rozmiar = 300) {
	int tmp;
	if (prawa - lewa < 1) return 0;
	int i = lewa;
	int klucz = tablica[prawa];
	for (int j = i; j < prawa; j++) {
		if (tablica[j] < klucz) {
			tmp = tablica[j];
			tablica[j] = tablica[i];
			tablica[i] = tmp;
			i++;
		}
	}
	tmp = tablica[prawa];
	tablica[prawa] = tablica[i];
	tablica[i] = tmp;
	sortowanieSzybkieTablica(tablica, lewa, i - 1, rozmiar);
	sortowanieSzybkieTablica(tablica, i + 1, prawa, rozmiar);
	return 1;
}

In [ ]:
void sortowanieSzybkieTablicaInicjacja(int* tablica, int rozmiar) {
	int res = sortowanieSzybkieTablica(tablica, 0, rozmiar - 1, rozmiar);
}

**Funkcja main**

In [ ]:
int main() {

	double BAczas = 0;
	double STczas = 0;
	double SZczas = 0;
	int iloscSortowan = 1000;
	int losowe[300];
    
	for (int i = 0; i < iloscSortowan; i++) {
		wypelnijLosowymi(losowe);

		int* BAtablica = wygenerujTablice(losowe);
		int* STtablica = wygenerujTablice(losowe);
		int* SZtablica = wygenerujTablice(losowe);

		BAczas += pomiar(BAtablica, 300, sortowanieBabelkoweTablica);
		STczas += pomiar(STtablica, 300, sortowanieStogoweTablica);
		SZczas += pomiar(SZtablica, 300, sortowanieSzybkieTablicaInicjacja);
	}
    
	printf("BA %lf ms\n", BAczas / iloscSortowan);
	printf("ST %lf ms\n", STczas / iloscSortowan);
	printf("SZ %lf ms\n", SZczas / iloscSortowan);
    
	return 0;

}